In [95]:
import matplotlib.pyplot as plt
import numpy as np
import umap.umap_ as umap

from sklearn.svm import SVC
from sklearn.cluster import SpectralClustering
from sklearn.metrics import normalized_mutual_info_score

from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, RealAmplitudes, EfficientSU2
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit.ml.datasets import digits

from qiskit.aqua.utils.dataset_helper import get_feature_dimension
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE

seed = 12345
algorithm_globals.random_seed = seed

In [96]:
digits = datasets.load_digits()

feature_dim = 2
n = 2
# flatten the images
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))
#X = tsne(X=data, no_dims=12, initial_dims=50, perplexity=30.0)
X = TSNE(n_components=feature_dim, method='exact').fit_transform(data)
equivariance_dim = 2
#X = umap.UMAP(random_state=42, n_components=equivariance_dim).fit_transform(data)
# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    X, digits.target, test_size=0.9, shuffle=False)

svc = SVC()
svc.fit(X_train,
        y_train)
score = svc.score(X_test, y_test)

print(f'Classic Result: {score}')

Classic Result: 0.8967861557478368


In [5]:
feature_map = ZZFeatureMap(
    feature_dimension=feature_dim,
    reps=2,
    entanglement='linear')
#circuit = EfficientSU(24, reps=1)
backend = QuantumInstance(
    BasicAer.get_backend('statevector_simulator'),
    shots=1024,
    seed_simulator=seed,
    seed_transpiler=seed)

kernel = QuantumKernel(feature_map=feature_map,
                       quantum_instance=backend)
svc_k = SVC(kernel=kernel.evaluate)

svc_k.fit(X_train,
        y_train)
score = svc_k.score(X_test, y_test)

print(f'With quantum kerel Result: {score}')

With quantum kerel Result: 0.10444993819530285


In [99]:
from qiskit.circuit.library import EfficientSU2
from qiskit.circuit.library import Permutation
from qiskit.circuit.library import RealAmplitudes

var_circuit = EfficientSU2(feature_dim, reps=2)
num_qubits = 2

variational_circ = RealAmplitudes(num_qubits, entanglement='full', reps=3)
circuit = EfficientSU2(12, reps=1)
perm = Permutation(4)
classifier =  ZZFeatureMap(2) + EfficientSU2(2)
#print(classifier)
pauli_feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=1, paulis = ['Z','X','ZY'])
#pauli_feature_map.draw()
kernel = QuantumKernel(
    feature_map=var_circuit,#pauli_feature_map,
    quantum_instance=backend
)

In [100]:
import seaborn as sns

matrix_train = kernel.evaluate(x_vec=X_train)
matrix_test = kernel.evaluate(x_vec=X_test, y_vec=X_train)

fig, axs = plt.subplots(1, 2, figsize=(10, 5), facecolor='w')
axs[0].imshow(np.asmatrix(matrix_train),
              interpolation='nearest', origin='upper', cmap='Blues')
axs[0].set_title("Quantum Training Kernel Matrix")
axs[1].imshow(np.asmatrix(matrix_test),
              interpolation='nearest', origin='upper', cmap='Reds')
axs[1].set_title("Quantum Test Kernel Matrix")
plt.show()

CircuitError: "ParameterVector par_x, ['par_x[0]', 'par_x[1]', 'par_x[2]', 'par_x[3]', 'par_x[4]', 'par_x[5]', 'par_x[6]', 'par_x[7]', 'par_x[8]', 'par_x[9]', 'par_x[10]', 'par_x[11]'] has length 12, which differs from value list [61.994972 40.324722] of len 2"

In [93]:
qsvc = QSVC(quantum_kernel=kernel)
qsvc.fit(X_train, y_train)
qsvc_score = qsvc.score(X_test, y_test)

print(f'QSVC classification test score: {qsvc_score}')

QSVC classification test score: 0.18046971569839307


In [33]:
import sys
!{sys.executable} -m pip uninstall umap --yes

Found existing installation: umap 0.1.1
Uninstalling umap-0.1.1:
  Successfully uninstalled umap-0.1.1


In [2]:
import sys
!{sys.executable} -m pip install qiskit_machine_learning

     |████████████████████████████████| 84 kB 1.6 MB/s eta 0:00:011
  Using cached sparse-0.12.0-py2.py3-none-any.whl (76 kB)
  Using cached numba-0.53.1-cp38-cp38-macosx_10_14_x86_64.whl (2.2 MB)
  Using cached llvmlite-0.36.0-cp38-cp38-macosx_10_9_x86_64.whl (18.5 MB)
You should consider upgrading via the '/Users/kslote/src/equivariance/quantum-svm/virt/bin/python -m pip install --upgrade pip' command.
